# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-01f70b5780-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA finals explorative search") 

Consider the following exploratory information need:

> investigate the world of basketball. Compare italian teams and US teams, country of citizenship of NBA players, and teams which won NBA finals.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36159`| Lebron James      | node |
| `wd:Q29545`| Kevin Durant      | node |





Also consider

```
wd:Q36159 ?p ?obj .
```

is the BGP to retrieve all **properties of Lebron James**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Identify the BGP for NBA finals

6. Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

8. Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

9. Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)



## Task 1
Identify the BGP for basketball player.

To find the BGP for basketball player, I have to:
 1. Find the property that links a person to being a basketball player
 2. Find the actuall object "Basket ball player"

In [4]:
# query example

# Let's explore the properties of Lebron James to find what makes him a baskketball player
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

# Let's dive deeper into the objects of the triples
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objname
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('name', 'family name'), ('obj', 'http://www.wikidata.org/entity/Q12188082'), ('objname', 'James')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q241329'), ('objname', '2006 FIBA World Championship')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for'), ('obj', 'http://www.wikidata.org/entity/Q680221'), ('objname', 'Laureus World Sports Award for Breakthrough of the Year')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('obj', 'http://www.wikidata.org/entity/Q1501719'), ('objname', 'J. Walter Kennedy Citizenship Award')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth'), ('obj', 'http://www.wikidata.org/entity/Q163132'), ('objname', 'Akron')]
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('name', 'residence'), ('obj', 'http://www.wikidata.org/entity/Q163132')

Looking at the results above, I can see that the property P106 named "occupation" links our player to the object Q3665646 named "basketball player". Then I make sure they're human for good mesure. Hence the final query:

Final query for this task

In [5]:
# write your final query
queryString = """
SELECT DISTINCT ?basketballPlayer ?name
WHERE {
   # bind something
   ?basketballPlayer wdt:P106 wd:Q3665646 ;
                     wdt:P31 wd:Q5 .
   ?basketballPlayer sc:name ?name
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('basketballPlayer', 'http://www.wikidata.org/entity/Q8192401'), ('name', 'Aitor Zárate')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q13708767'), ('name', 'Aristide Landi')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q15477486'), ('name', 'Małgorzata Janowicz')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q15737093'), ('name', 'Jarrett Durham')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q15843419'), ('name', 'Ryan Brooks')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q15850750'), ('name', 'Thomas Hering')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q15856626'), ('name', 'Ama Degbeon')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q15870814'), ('name', 'Josef Klíma')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q15879801'), ('name', 'Niels Vorenhout')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q15881370'), ('name', 'Roel Aarts')]
[('basketballPlayer', 'http://www.wikidata.org/entity

## Task 2
Identify the BGP for basketball.

In [6]:
# write your queries

# Query to find entities that are subclasses of entities with "basketball in their name"
queryString = """
SELECT DISTINCT ?game ?nameg
WHERE {
   # bind something
   ?game wdt:P279 ?obj ;
         sc:name ?nameg .
   FILTER regex(?nameg, "basketball", "i")
}
LIMIT 20
"""

# One of the results of the previus is named "men's college basketball" and its IRI is Q101003088. Let's explore its parent classes
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q101003088 wdt:P279 ?obj .
      ?obj   sc:name ?name .
}
LIMIT 10
"""

# One of the results of the previus is named "college basketball" and its IRI is Q48890. Let's explore its parent classes
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q48890 wdt:P279 ?obj .
      ?obj   sc:name ?name .
}
LIMIT 10
"""

# We dound the entity basketball with IRI Q5372. Let's explore it's propeties
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q5372 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

# We can see that basketball is a subclass of ball game IRI Q877517. let;s find all subclasses of ball game

queryString = """
SELECT DISTINCT ?basketball ?name
WHERE {
   # bind something
   ?basketball wdt:P279 wd:Q877517 .
   # get the label
   ?basketball sc:name ?name.
}
LIMIT 20
"""

# Now we filter to find basketball

queryString = """
SELECT DISTINCT ?basketball ?name
WHERE {
   # bind something
   ?basketball wdt:P279 wd:Q877517 .
   # get the label
   ?basketball sc:name ?name.
   FILTER regex(?name, ".*basketball.*", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('basketball', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
1


Final query for this task

In [7]:
# write your final query
queryString = """
SELECT DISTINCT ?basketball ?name
WHERE {
   # bind something
   ?basketball wdt:P279 wd:Q877517 .
   # get the label
   ?basketball sc:name ?name.
   FILTER regex(?name, ".*basketball.*", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('basketball', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
1


## Task 3
Identify the BGP for basketball team.

In [8]:
# write your queries
# Looking back at our query to explor a basketball player's properties
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objname
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objname.
}
LIMIT 20
"""

# Let's look at the occupations

queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   wd:Q36159 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?objname.
}
LIMIT 20
"""

# Now the occupation basketball player's properties

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objname
WHERE {
   # bind something
   wd:Q3665646 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objname.
}
LIMIT 20 
"""

# We can see it has a property "part of" and it's a part of "basketball team" IRI Q13393265. let's find all instances of basketball team

queryString = """
SELECT DISTINCT ?team ?name
WHERE {
   # bind something
   ?team  wdt:P31 wd:Q13393265 .
   # get the label
   ?team sc:name ?name .
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q8206968'), ('name', 'Hebraica y Macabi')]
[('team', 'http://www.wikidata.org/entity/Q5144784'), ('name', "Colgate Raiders men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q7051115'), ('name', "Norfolk State Spartans men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q7397248'), ('name', "Sacred Heart Pioneers men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q15725839'), ('name', 'A.S. Kentavros Neas Kallikrateias')]
[('team', 'http://www.wikidata.org/entity/Q107876527'), ('name', "Iowa Western Reivers men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q15927936'), ('name', 'Westchester Knicks')]
[('team', 'http://www.wikidata.org/entity/Q810271'), ('name', 'BC Zepter Vienna')]
[('team', 'http://www.wikidata.org/entity/Q100320348'), ('name', "East Central Tigers men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q16201949'), ('name', "DePaul Blue Demons women's basketball")]
[('

Final query for this task

In [9]:
# write your final query
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
   # bind something
   ?team  wdt:P31 wd:Q13393265 .
   # get the label
   ?team sc:name ?name .
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q8206968'), ('name', 'Hebraica y Macabi')]
[('team', 'http://www.wikidata.org/entity/Q5144784'), ('name', "Colgate Raiders men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q7051115'), ('name', "Norfolk State Spartans men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q7397248'), ('name', "Sacred Heart Pioneers men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q15725839'), ('name', 'A.S. Kentavros Neas Kallikrateias')]
[('team', 'http://www.wikidata.org/entity/Q107876527'), ('name', "Iowa Western Reivers men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q15927936'), ('name', 'Westchester Knicks')]
[('team', 'http://www.wikidata.org/entity/Q810271'), ('name', 'BC Zepter Vienna')]
[('team', 'http://www.wikidata.org/entity/Q100320348'), ('name', "East Central Tigers men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q16201949'), ('name', "DePaul Blue Demons women's basketball")]
[('

## Task 4
Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

In [10]:
# write your queries
# Let's look at all the properties of a team
queryString = """
SELECT DISTINCT  ?p ?pName
WHERE {
   # bind something
   ?team  wdt:P31 wd:Q13393265 ;
          ?p ?obj .
   # get the label
   ?p sc:name ?pName .
}
LIMIT 20 
"""

# We can see a property named country IRI P17. We can filter for italy.
# It also has property inception IRI P571, we can use it to filter by date
queryString = """
SELECT DISTINCT  ?team
WHERE {
   # bind something
   ?team  wdt:P31 wd:Q13393265 ;
          wdt:P17 wd:Q38 ;
          wdt:P571 ?inceptionYear ;
          ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   FILTER (YEAR(?inceptionYear) < 1945)
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q1139737')]
[('team', 'http://www.wikidata.org/entity/Q36362')]
[('team', 'http://www.wikidata.org/entity/Q36500')]
[('team', 'http://www.wikidata.org/entity/Q36264')]
[('team', 'http://www.wikidata.org/entity/Q36311')]
[('team', 'http://www.wikidata.org/entity/Q36572')]
[('team', 'http://www.wikidata.org/entity/Q3891980')]
[('team', 'http://www.wikidata.org/entity/Q7127695')]
[('team', 'http://www.wikidata.org/entity/Q3754192')]
[('team', 'http://www.wikidata.org/entity/Q16639152')]
[('team', 'http://www.wikidata.org/entity/Q961782')]
[('team', 'http://www.wikidata.org/entity/Q3972903')]
[('team', 'http://www.wikidata.org/entity/Q3626684')]
[('team', 'http://www.wikidata.org/entity/Q3747654')]
14


Final query for this task

In [11]:
# write your final query
queryString = """
SELECT DISTINCT  ?team ?name YEAR(?inceptionYear) as ?yearOfInception
WHERE {
   # bind something
   ?team  wdt:P31 wd:Q13393265 ;
          wdt:P17 wd:Q38 ;
          wdt:P571 ?inceptionYear .
   # get the label
   ?team sc:name ?name .
   FILTER (YEAR(?inceptionYear) < 1945)
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q3891980'), ('name', 'Nuova Pallacanestro Pavia'), ('yearOfInception', '1943')]
[('team', 'http://www.wikidata.org/entity/Q1139737'), ('name', 'Fortitudo Pallacanestro Bologna'), ('yearOfInception', '1932')]
[('team', 'http://www.wikidata.org/entity/Q36362'), ('name', 'Olimpia Milano'), ('yearOfInception', '1936')]
[('team', 'http://www.wikidata.org/entity/Q36500'), ('name', 'Mens Sana Basket'), ('yearOfInception', '1861')]
[('team', 'http://www.wikidata.org/entity/Q36264'), ('name', 'Virtus Pallacanestro Bologna'), ('yearOfInception', '1927')]
[('team', 'http://www.wikidata.org/entity/Q36311'), ('name', 'Pallacanestro Cantù'), ('yearOfInception', '1936')]
[('team', 'http://www.wikidata.org/entity/Q7127695'), ('name', 'Pallacanestro Pavia'), ('yearOfInception', '1943')]
[('team', 'http://www.wikidata.org/entity/Q3754192'), ('name', 'Fulgor Omegna'), ('yearOfInception', '1937')]
[('team', 'http://www.wikidata.org/entity/Q36572'), ('name'

## Task 5
Identify the BGP for NBA finals

In [12]:
# write your queries

# This query is to find some NBA finals entities to explore
# Its output is the IRI and lable of the subject, predicate, and object respectivly, filtered for objects that have "NBA finals" in their lable
queryString = """
SELECT DISTINCT  ?s ?sName ?p ?pName ?obj ?objName
WHERE {
   # bind something
   ?s ?p ?obj .
   ?obj sc:name ?objName .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   FILTER regex(?objName, ".*NBA final.*", "i")
}
LIMIT 20 
"""

#The output of the previous query contains the 2017 NBA Finals IRI Q29096468. Let's explore it's properties
# WE can see that the 2017 NBA Final is an instance of NBA Finals IRI Q842375. 
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q29096468 ?p ?obj .
   ?obj sc:name ?objName .
   ?p sc:name ?pName .
}
LIMIT 20 
"""

# Finding all the instances of NBA Finals is the BGP for NBA finals
# The output contains some other finals that we don't want. We need to add a filter
queryString = """
SELECT DISTINCT ?final ?name
WHERE {
   # bind something
   ?final wdt:P31 wd:Q842375 .
   ?final sc:name ?name .
}
LIMIT 20 
"""

# Adding the filter for anual NBA finals only

queryString = """
SELECT DISTINCT ?final ?name
WHERE {
   # bind something
   ?final wdt:P31 wd:Q842375 .
   ?final sc:name ?name .
   FILTER regex(?name, ".*NBA final.*", "i")
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('final', 'http://www.wikidata.org/entity/Q24264147'), ('name', '2016 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q29096468'), ('name', '2017 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4588662'), ('name', '1993 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4598928'), ('name', '2001 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4589592'), ('name', '1994 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4590500'), ('name', '1995 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q2036161'), ('name', '2008 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q170590'), ('name', '2006 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4600307'), ('name', '2002 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q2552745'), ('name', '2007 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4584557'), ('name', '1988 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4587787'), ('name', '19

Final query for this task

In [13]:
# write your final query
queryString = """
SELECT DISTINCT ?final ?name
WHERE {
   # bind something
   ?final wdt:P31 wd:Q842375 .
   ?final sc:name ?name .
   FILTER regex(?name, ".*NBA final.*", "i")
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('final', 'http://www.wikidata.org/entity/Q24264147'), ('name', '2016 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q29096468'), ('name', '2017 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4588662'), ('name', '1993 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4598928'), ('name', '2001 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4589592'), ('name', '1994 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4590500'), ('name', '1995 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q2036161'), ('name', '2008 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q170590'), ('name', '2006 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4600307'), ('name', '2002 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q2552745'), ('name', '2007 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4584557'), ('name', '1988 NBA Finals')]
[('final', 'http://www.wikidata.org/entity/Q4587787'), ('name', '19

## Task 6
Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

In [81]:
# write your queries

# Let's explore the properties of Lebron James to find what links him a baskketball team
# The property "member of sports team" IRI P54 seems like what we're looking for. Let's explre it further
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

# Exploring the property "member of sports team" IRI P54
# P54 is the property we're looking for
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q36159 wdt:P54 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 100
"""

#Now we can look for the basketballl teams where Lebron James and Kevin Durant played
queryString = """
SELECT DISTINCT  ?team ?name
WHERE {
   # bind something
   {wd:Q36159 wdt:P54 ?team} UNION
   {wd:Q29545 wdt:P54 ?team} .
   # get the label
   ?team sc:name ?name.
}
LIMIT 100
"""

# We need to find the property that tells us that a team has won an nba final
queryString = """
SELECT DISTINCT  ?p ?pName
WHERE {
   # bind something
   {wd:Q36159 wdt:P54 ?team} UNION
   {wd:Q29545 wdt:P54 ?team} .
   ?team ?p ?obj .
   # get the label
   ?p sc:name ?pName.
}
LIMIT 100
"""

# The property "award received" IRI P166 from the last query looks promessing, let's explore it. 
queryString = """
SELECT DISTINCT  ?obj ?objName
WHERE {
   # bind something
   {wd:Q36159 wdt:P54 ?team} UNION {wd:Q29545 wdt:P54 ?team} .
   ?team wdt:P166 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 100
"""

# The results are not correct. Which leads me to believe that this property is not gonna give us what we want. 
queryString = """
SELECT DISTINCT  ?team ?name count(?award) as ?victories
WHERE {
   # bind something
   {wd:Q36159 wdt:P54 ?team} UNION {wd:Q29545 wdt:P54 ?team} .
   optional {?team wdt:P166 ?award .}
   # get the label
   ?team sc:name ?name.
}
GROUP BY ?team  ?name
LIMIT 100
"""

# Let's explore triples where basketbball teams are the objects. i'll use Cleveland Cavaliers (Q162990) from the first query
queryString = """
SELECT DISTINCT  ?p ?name
WHERE {
   # bind something
   ?s ?p wd:Q162990 .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

# We can see the property "winnir" IRI P1346 from the last query. let's check the subject of the triple.
queryString = """
SELECT DISTINCT  ?s ?name
WHERE {
   # bind something
   ?s wdt:P1346 wd:Q162990 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

#It's the 2015–16 NBA season. The property "winner" is what we need. Let's find all NBA winners
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
   # bind something
   ?s wdt:P1346 ?team .
   # get the label
   ?s sc:name ?season.
   ?team sc:name ?name.
   FILTER regex(?season, "[0-9] NBA Finals")
}
LIMIT 100
"""

#Great! Now we can count the victories of teams where Lebron James and Kevin Durant played
queryString = """
SELECT DISTINCT  ?team ?name count(?s) as ?victories
WHERE {
   # bind something
   {wd:Q36159 wdt:P54 ?team} UNION {wd:Q29545 wdt:P54 ?team} .
   OPTIONAL {?s wdt:P1346 ?team. 
             ?s sc:name ?sName.
             FILTER regex(?sName, "[0-9] NBA Finals")} .
   # get the label
   ?team sc:name ?name.
}
GROUP BY ?team ?name
LIMIT 100
"""


print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q157376'), ('name', 'Golden State Warriors'), ('victories', '1')]
[('team', 'http://www.wikidata.org/entity/Q244847'), ('name', "United States men's national basketball team"), ('victories', '0')]
[('team', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers'), ('victories', '6')]
[('team', 'http://www.wikidata.org/entity/Q572134'), ('name', 'Brooklyn Nets'), ('victories', '0')]
[('team', 'http://www.wikidata.org/entity/Q169138'), ('name', 'Miami Heat'), ('victories', '3')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('name', 'Seattle SuperSonics'), ('victories', '0')]
[('team', 'http://www.wikidata.org/entity/Q180950'), ('name', 'Oklahoma City Thunder'), ('victories', '0')]
[('team', 'http://www.wikidata.org/entity/Q162990'), ('name', 'Cleveland Cavaliers'), ('victories', '0')]
[('team', 'http://www.wikidata.org/entity/Q7707830'), ('name', "Texas Longhorns men's basketball"), ('victories', '0')]
9


Final query for this task

In [82]:
# write your final query
queryString = """
SELECT DISTINCT  ?team ?name count(?s) as ?victories
WHERE {
   # bind something
   {wd:Q36159 wdt:P54 ?team} UNION {wd:Q29545 wdt:P54 ?team} .
   OPTIONAL {?s wdt:P1346 ?team. 
             ?s sc:name ?sName.
             FILTER regex(?sName, "[0-9] NBA Finals")} .
   # get the label
   ?team sc:name ?name.
}
GROUP BY ?team ?name
LIMIT 100
"""


print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q157376'), ('name', 'Golden State Warriors'), ('victories', '1')]
[('team', 'http://www.wikidata.org/entity/Q244847'), ('name', "United States men's national basketball team"), ('victories', '0')]
[('team', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers'), ('victories', '6')]
[('team', 'http://www.wikidata.org/entity/Q572134'), ('name', 'Brooklyn Nets'), ('victories', '0')]
[('team', 'http://www.wikidata.org/entity/Q169138'), ('name', 'Miami Heat'), ('victories', '3')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('name', 'Seattle SuperSonics'), ('victories', '0')]
[('team', 'http://www.wikidata.org/entity/Q180950'), ('name', 'Oklahoma City Thunder'), ('victories', '0')]
[('team', 'http://www.wikidata.org/entity/Q162990'), ('name', 'Cleveland Cavaliers'), ('victories', '0')]
[('team', 'http://www.wikidata.org/entity/Q7707830'), ('name', "Texas Longhorns men's basketball"), ('victories', '0')]
9


## Task 7
Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

In [7]:
# write your queries
# let's find teasm that play in the NBA. To find the property that makes the assosiation, 
#I'll explore the properties of the Los Angeles Lakers Q121783
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q121783 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

# The "league" (P118) seems promessing. Let's chek it's objects
# We find the entity National Basketball Association Q155223
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q121783 wdt:P118 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

# Now we can find NBA teams using the entity National Basketball Association Q155223
queryString = """
SELECT DISTINCT ?team ?name ?playerName
WHERE {
   # bind something
   ?team  wdt:P31 wd:Q13393265 ;
          wdt:P118 wd:Q155223 .
   # get the label
   ?team sc:name ?name .
}
order by ?name
LIMIT 20 
"""

# And then we can find the players that played for those teams

queryString = """
SELECT DISTINCT ?team ?name ?playerName
WHERE {
   # bind something
   ?team  wdt:P31 wd:Q13393265 ;
          wdt:P118 wd:Q155223 .
   ?basketballPlayer wdt:P106 wd:Q3665646 ;
                     wdt:P31 wd:Q5 ;
                     wdt:P54 ?team .
   ?basketballPlayer sc:name ?playerName .
   # get the label
   ?team sc:name ?name .
}
order by ?name
LIMIT 20 
"""

# Now we need to find exah players nationality. In our previuos tasks we found the propert "country of citizenship" IRI P27.
# But we need to find the continent of each player
queryString = """
SELECT DISTINCT ?basketballPlayer ?name ?country ?countryName
WHERE {
   # bind something
   ?basketballPlayer wdt:P106 wd:Q3665646 ;
                     wdt:P31 wd:Q5 .
   ?basketballPlayer wdt:P27 ?country.
   ?basketballPlayer sc:name ?name .
   ?country sc:name ?countryName.
}
LIMIT 20
"""

# I'll explore the properties of a country to find it's continent

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   ?basketballPlayer wdt:P106 wd:Q3665646 ;
                     wdt:P31 wd:Q5 .
   ?basketballPlayer wdt:P27 ?country.
   ?country ?p ?obj .
   ?basketballPlayer sc:name ?name .
   ?country sc:name ?countryName.
   ?p sc:name ?pName.
   ?country sc:name ?countryName.
   FILTER regex(?pName, ".*continent.*", "i")
}
LIMIT 20
"""

#Perfect! the country has a "continent" property IRI P30. Let's use it to find the continent of each of the players
queryString = """
SELECT DISTINCT ?basketballPlayer ?name ?continent ?continentName
WHERE {
   # bind something
   ?basketballPlayer wdt:P106 wd:Q3665646 ;
                     wdt:P31 wd:Q5 .
   ?basketballPlayer wdt:P27 ?country.
   ?country wdt:P30 ?continent .
   ?basketballPlayer sc:name ?name .
   ?continent sc:name ?continentName.
}
LIMIT 20
"""

#Now we cancount the players of each continent
queryString = """
SELECT DISTINCT ?continent ?continentName count(?basketballPlayer) as ?players
WHERE {
   # bind something
   ?basketballPlayer wdt:P106 wd:Q3665646 ;
                     wdt:P31 wd:Q5 .
   ?basketballPlayer wdt:P27 ?country.
   ?country wdt:P30 ?continent .
   ?basketballPlayer sc:name ?name .
   ?continent sc:name ?continentName.
}
GROUP BY ?continent ?continentName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('players', '1405')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('players', '13853')]
[('continent', 'http://www.wikidata.org/entity/Q27611'), ('continentName', 'Central America'), ('players', '1')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('players', '14669')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('players', '14538')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('continentName', 'Eurasia'), ('players', '505')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('players', '4309')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('players', '1368')]
8


Final query for this task

In [ ]:
# write your final query
queryString = """
SELECT DISTINCT ?continent ?continentName count(?basketballPlayer) as ?players
WHERE {
   # bind something
   ?basketballPlayer wdt:P106 wd:Q3665646 ;
                     wdt:P31 wd:Q5 .
   ?basketballPlayer wdt:P27 ?country.
   ?country wdt:P30 ?continent .
   ?basketballPlayer sc:name ?name .
   ?continent sc:name ?continentName.
}
GROUP BY ?continent ?continentName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

## Task 8
Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

In [17]:
# write your queries
# Let's explore the properties of Lebron James to find if he's ever won the award
# We have a "award received" IRI P166. We can explore it
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

# Exploring the "award received" for Lebron James.
#lebron james has won the award and it's IRI is Q222047
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P166 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

# Let's look at what type of award it is
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q222047 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

# let's write the final query
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
   # bind something
   ?award wdt:P31 wd:Q652965 .
   # get the label
   ?award sc:name ?name.
   FILTER regex(?name, "^NBA Most Valuable Player Award", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q222047'), ('name', 'NBA Most Valuable Player Award')]
1


Final query for this task

In [18]:
# write your final query
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
   # bind something
   ?award wdt:P31 wd:Q652965 .
   # get the label
   ?award sc:name ?name.
   FILTER regex(?name, "^NBA Most Valuable Player Award", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q222047'), ('name', 'NBA Most Valuable Player Award')]
1


## Task 9
Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)

In [46]:
# write your queries
#Let's find all the instances of a player winnig the award
queryString = """
SELECT ?basketballPlayer ?name ?award ?awardName
WHERE {
   # bind something
   ?basketballPlayer wdt:P106 wd:Q3665646 ;
                     wdt:P31 wd:Q5 ;
                     wdt:P166 ?award .
   ?award wdt:P31 wd:Q652965 . 
   ?basketballPlayer sc:name ?name .
   ?award sc:name ?awardName .
   FILTER regex(?awardName, "^NBA Most Valuable Player Award", "i")
}
ORDER BY ?name
LIMIT 30
"""

#now we can count the victories
queryString = """
SELECT ?basketballPlayer ?name count(?award) as ?victories
WHERE {
   # bind something
   ?basketballPlayer wdt:P106 wd:Q3665646 ;
                     wdt:P31 wd:Q5 ;
                     wdt:P166 ?award .
   ?award wdt:P31 wd:Q652965 . 
           ?basketballPlayer sc:name ?name .
   ?award sc:name ?awardName .
   FILTER regex(?awardName, "^NBA Most Valuable Player Award", "i")
}
GROUP BY ?basketballPlayer ?name 
ORDER BY ?name
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('basketballPlayer', 'http://www.wikidata.org/entity/Q190094'), ('name', 'Allen Iverson'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q242185'), ('name', 'Bill Russell'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q363246'), ('name', 'Bill Walton'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q312643'), ('name', 'Bob Cousy'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q553232'), ('name', 'Bob McAdoo'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q368561'), ('name', 'Bob Pettit'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q192707'), ('name', 'Charles Barkley'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q264393'), ('name', 'Dave Cowens'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q211876'), ('name', 'David Robinson'), ('victories', '1

Final query for this task

In [48]:
# write your final query
queryString = """
SELECT ?basketballPlayer ?name count(?award) as ?victories
WHERE {
   # bind something
   ?basketballPlayer wdt:P106 wd:Q3665646 ;
                     wdt:P31 wd:Q5 ;
                     wdt:P166 ?award .
   ?award wdt:P31 wd:Q652965 . 
           ?basketballPlayer sc:name ?name .
   ?award sc:name ?awardName .
   FILTER regex(?awardName, "^NBA Most Valuable Player Award", "i")
}
GROUP BY ?basketballPlayer ?name 
ORDER BY ?name
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('basketballPlayer', 'http://www.wikidata.org/entity/Q190094'), ('name', 'Allen Iverson'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q242185'), ('name', 'Bill Russell'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q363246'), ('name', 'Bill Walton'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q312643'), ('name', 'Bob Cousy'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q553232'), ('name', 'Bob McAdoo'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q368561'), ('name', 'Bob Pettit'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q192707'), ('name', 'Charles Barkley'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q264393'), ('name', 'Dave Cowens'), ('victories', '1')]
[('basketballPlayer', 'http://www.wikidata.org/entity/Q211876'), ('name', 'David Robinson'), ('victories', '1